# #4 Intro to Python

In [ ]:
from toImportLibraries import *
from toImportFunctions import *
checkSciLibVersions()

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

## List Comprehensions 
$ S=\left\{ x^{2}:x\:in\:\left\{ 0\ldots10\right\} \right\} $

$ V=\left\{ 1,2,4,8,\ldots,2^{12}\right\} $

$ M=\left\{ x\:|\:x\:in\:S\:and\:x\:even\right\} $

In [ ]:
S = [x**2 for x in range(11)]
V = [2**i for i in range(13)]
M = [x for x in S if x % 2 ==0]

print('S=', S, '\nV=', V, '\nM=', M)

## A small simulation example

**Let's see a Brownian bridge**

In [ ]:
class Parameters:
    tMax = 1000
    T = 1.0
    M  = 100000.0
    AT = 0.0
    BT = 0.0
    numOfPaths = 100
    conf = 0.95
    
    def printValues():
        pass

In [ ]:
p = Parameters
p.tMax = 400
p.T = 1.0
p.numOfPaths = 137

**Wiener Process**

In [ ]:
def initWProcess(p):
    dfW0 = pd.DataFrame(np.array([np.ones(p.numOfPaths)*0.0]))
    dfW = pd.DataFrame(np.random.randn(p.tMax, p.numOfPaths)/np.sqrt(365))
    dfW = pd.concat([dfW0,dfW])
    dfW.reset_index(inplace=True, drop=True)
    dfW = dfW.cumsum()
    
    dfW['t'] = range(0,len(dfW))
    dfW['t'] = dfW['t'] * 1.0
    dfW['t'] = dfW['t'] / 365
    
    return dfW

In [ ]:
V = initWProcess(p)
V.head()

[Matplotlib](https://matplotlib.org/stable/gallery/index.html) for more examples

In [ ]:
plt.figure(figsize=(20,10), facecolor='white')
plt.title('random walk')
plt.xlabel('time [d]')
plt.ylabel('dist')

plt.plot(V.iloc[:,0:p.numOfPaths], color='k', alpha=0.1)
plt.show()

**[Brownian bridge](https://en.wikipedia.org/wiki/Brownian_bridge)**
$$B(t) = W(t) - \frac{t}{T} W(T)$$

In [ ]:
def BrownianBridge(row, WT, T):
    #print(row.iloc[0],'XXX',row.iloc[1], row)
    x = row.iloc[0]-WT*row.iloc[1]/T
    return x

V = initWProcess(p)
for c in V.columns[:p.numOfPaths]:
    #print(c, end='$$$')
    WT = V[V['t']==p.T][c].iloc[0]
    V[c] = V[[c,'t']].apply(lambda x: BrownianBridge(x,WT,p.T), axis=1)

In [ ]:
def getPercentile(row,p):
    return np.percentile(row, p)

def getGaussConf(row,ci):
    n = len(row)
    return sp.stats.t.interval(0.95, len(row)-1, loc=np.mean(row), scale=sp.stats.sem(row))
    #return np.abs(sp.stats.t.ppf((1-ci)/2,len(row)-1) / np.sqrt(len(row)))


In [ ]:
#add some stats
V['mean'] = V[V.columns[:p.numOfPaths]].mean(axis=1)
V['stddev'] = V[V.columns[:p.numOfPaths]].std(axis=1)
V['97th'] = V[V.columns[:p.numOfPaths]].apply(lambda x: getPercentile(x,97),axis=1)
V['03th'] = V[V.columns[:p.numOfPaths]].apply(lambda x: getPercentile(x,3),axis=1)

V['GaussConf97B'] = V[V.columns[:p.numOfPaths]].apply(lambda x: getGaussConf(x,0.97)[0], axis=1)
V['GaussConf97B'].fillna(0, inplace=True)
V['GaussConf97T'] = V[V.columns[:p.numOfPaths]].apply(lambda x: getGaussConf(x,0.97)[1], axis=1)
V['GaussConf97T'].fillna(0, inplace=True)

#V['GaussConf97'] = V[V.columns[:p.numOfPaths]].apply(lambda x: getGaussConf(x,0.97), axis=1)
#V['GaussConf97'].fillna(0, inplace=True)
#V['GaussConf97T'] = V['mean'] + V['stddev']*V['GaussConf97']
#V['GaussConf97B'] = V['mean'] - V['stddev']*V['GaussConf97']

In [ ]:
plt.figure(figsize=(20,10), facecolor='white')
plt.title('Borwnian Bridge')
plt.xlabel('time [d]')
plt.ylabel('dist')

plt.plot(V.iloc[:,0:p.numOfPaths], color='k', alpha=0.1)
plt.plot(V['mean'], color='r', alpha=1, label='mean')
plt.plot(V['stddev'], color='g', alpha=1, label='std')
plt.plot(-V['stddev'], color='g', alpha=1, label='')
plt.plot(V['97th'], color='b', alpha=1, label='3/97th')
plt.plot(V['03th'], color='b', alpha=1, label='')
plt.plot(V['GaussConf97B'], color='r', alpha=0.4, label='GaussConfInt.97')
plt.plot(V['GaussConf97T'], color='r', alpha=0.4, label='')
plt.legend(loc='upper left')
plt.show()

## Some plots

In [ ]:
import matplotlib.ticker as mtick
df = pd.DataFrame({'x': [1000,2000,3000,4000], 'y1': [100,120,110,89], 'y2': [100,105,80,100], 'y3': [100,80,90,120]})

fig = plt.figure(figsize=(8,5))

ax = fig.add_subplot()
ax.plot(df['x'], df['y1']/100, label="test1", c='orange', marker='o')
ax.plot(df['x'], df['y2']/100, label="test2", c='tab:blue', marker='o')
ax.plot(df['x'], df['y3']/100, label="test3", c='gray', marker='o')
ax.set_title('Title')
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())
vals = ax.get_yticks()
ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])
vals = [1000,2000,3000,4000]
#ax.set_xticklabels(vals)
plt.xticks(np.arange(1000, 5000, 1000))
#ax.set_ylabel('%')

# Place a legend above this subplot, expanding itself to
# fully use the given bounding box.

ax.grid(axis='y')

ax.legend(bbox_to_anchor=(0., -.2, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0.)

plt.show()

## Creating Power Point slides

In [ ]:
from pptx import Presentation     
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.chart.data import ChartData
from pptx.util import Inches

imgpth = 'herbStock.png'
  
prs = Presentation()
    # Use the output from analyze_ppt to understand which layouts and placeholders
    # to use
    # Create a title slide first
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]
title.text = "Quarterly Report"
subtitle.text = "Generated on: {:%Y.%m.%d.}".format(date.today())

graph_slide_layout = prs.slide_layouts[1]
slide = prs.slides.add_slide(graph_slide_layout)
title = slide.shapes.title
title.text = "Herbs"
placeholder = slide.placeholders[1]
left = Inches(1)
top = Inches(2)
pic = slide.shapes.add_picture(imgpth,left, top, width=Inches(6))

prs.save("Output.pptx")

## Dealing with Excel files #1 - [openpyxl](https://openpyxl.readthedocs.io/en/stable/usage.html)

In [ ]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

wb = Workbook()

dest_filename = 'empty_book.xlsx'

ws1 = wb.active
ws1.title = "range names"

for row in range(1, 40):
    ws1.append(range(600))

ws2 = wb.create_sheet(title="Pi")

ws2['F5'] = 3.14

ws3 = wb.create_sheet(title="Data")
for row in range(10, 20):
    for col in range(27, 54):
        _ = ws3.cell(column=col, row=row, value="{0}".format(get_column_letter(col)))
print(ws3['AA10'].value)

wb.save(filename = dest_filename)

In [ ]:
from openpyxl import load_workbook
wb = load_workbook(filename = 'empty_book.xlsx')
sheet_ranges = wb['range names']
print(sheet_ranges['D18'].value)

## Dealing with Excel files #2 - [xlwings](https://docs.xlwings.org/en/stable/quickstart.html)

In [ ]:
import xlwings as xw
wb = xw.Book('empty_book.xlsx')

In [ ]:
sheet = wb.sheets['Data']

In [ ]:
sheet.range('A1').value = 'Foo 1'
sheet.range('A1').value

In [ ]:
df = pd.DataFrame([[1,2], [3,4]], columns=['a', 'b'])
sheet.range('A1').value = df
sheet.range('A1').options(pd.DataFrame, expand='table').value